In [ ]:
%matplotlib inline
from __future__ import (
    print_function,
    division,
    unicode_literals
)

import itertools as it

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as skl

matplotlib.style.use('ggplot')

In [ ]:
df = pd.read_csv('DMC_2015_orders_train.txt', delimiter='|')
print(', '.join(df.columns))

In [ ]:
# Datetime
df.orderTime = pd.to_datetime(df.orderTime)
df.couponsReceived = pd.to_datetime(df.couponsReceived)
df['deltaT'] = (df.orderTime - df.couponsReceived)

# Brands
brands = reduce(lambda acc, x: acc.union(set(df['brand{}'.format(x)])), (1, 2, 3), set())
    
for num, brand in it.product((1, 2, 3), brands):
    df['brand{}_{}'.format(num, brand)] = 0
    
for num, row in df.iterrows():
    df.loc[num, 'brand1_{}'.format(row.brand1)] = 1
    df.loc[num, 'brand2_{}'.format(row.brand2)] = 1
    df.loc[num, 'brand3_{}'.format(row.brand3)] = 1

In [ ]:
sig = df[df.coupon1Used == 1]
bkg = df[df.coupon1Used == 0]

plt.hist([float(t) for t in sig.deltaT.values], alpha=0.5, normed=True)
plt.hist([float(t) for t in bkg.deltaT.values], alpha=0.5, normed=True)
_ = 1

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)

#bdt.fit(X, y)

In [ ]:
a = df.brand1.astype('category')
b = df.brand2.astype('category')
c = df.brand3.astype('category')
len(set(a.unique()).union(b.unique()).union(c.unique()))

In [ ]:
corr = df.corr()
plt.imshow(corr, cmap='seismic', vmin=-1, vmax=1, interpolation='none')
plt.colorbar()
plt.xticks(np.arange(0, len(corr.columns)), corr.columns, rotation='vertical')
plt.yticks(np.arange(0, len(corr.columns)), corr.columns)
_ = 1
#plt.tight_layout()
#plt.savefig('corr.png', bbox_inches='tight')